In [1]:
import sys; sys.path.append("../../automl/")

In [2]:
from pathlib import Path
import yaml

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from src.automl.model import AutoML
from src.automl.model.metrics import RocAuc

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'nltk' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
[2024-11-05 15:30:39]
/home/peter/venvs/base_venv/lib/python3.10/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/

## Constants

In [3]:
RANDOM_SEED = 77
DATA_PATH = Path("../../data/")
CONFIG_PATH = Path("../../configs/config.yaml")

In [4]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

## Data

In [5]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc.parquet")
df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [6]:
ohe_cols = df_train.columns[df_train.columns.str.startswith("OneHotEncoder")].values.tolist()
oe_cols = df_train.columns[df_train.columns.str.startswith("OrdinalEncoder")].values.tolist()
te_cols = df_train.columns[df_train.columns.str.startswith("MeanTargetEncoder")].values.tolist()

In [7]:
# take ordinal encoded columns
X_train, y_train = df_train[cfg["selected_features"] + oe_cols + ohe_cols], df_train["target"]
X_test, y_test = df_test[cfg["selected_features"] + oe_cols + ohe_cols], df_test["target"]

In [8]:
display(y_train.value_counts(normalize=True))
display(y_test.value_counts(normalize=True))

0    0.940853
1    0.059147
Name: target, dtype: float64

0    0.941322
1    0.058678
Name: target, dtype: float64

In [9]:
categorical_features = ohe_cols + oe_cols

## Model

In [10]:
model = AutoML("classification", RocAuc(), n_jobs=16, random_state=RANDOM_SEED, tuning_timeout=60 * 5)

In [11]:
model.fit(X_train, y_train, X_test, y_test, categorical_features=categorical_features, save_models=False, save_test=False)

[2024-11-05 12:57:58,736] - [   MODEL    ] - 1 out of 15. LogisticRegression
[2024-11-05 12:57:58,739] - [   START    ] - Working with LogisticRegression
[2024-11-05 12:57:58,740] - [   START    ] - Tuning LogisticRegression
[2024-11-05 12:58:18,447] - [BEST PARAMS ] - {'C': 0.005994842503189409, 'class_weight': 'balanced', 'max_iter': 1000, 'n_jobs': 16, 'random_state': 77}
[2024-11-05 12:58:18,448] - [    END     ] - Tuning LogisticRegression
[2024-11-05 12:58:18,449] - [   START    ] - Fitting LogisticRegression
[2024-11-05 12:58:18,526] - [    FIT     ] - LogisticRegression fold 0
[2024-11-05 12:58:21,700] - [    FIT     ] - LogisticRegression fold 1
[2024-11-05 12:58:24,292] - [    FIT     ] - LogisticRegression fold 2
[2024-11-05 12:58:27,107] - [    FIT     ] - LogisticRegression fold 3
[2024-11-05 12:58:29,872] - [    FIT     ] - LogisticRegression fold 4
[2024-11-05 12:58:32,666] - [    END     ] - Fitting LogisticRegression
[2024-11-05 12:58:33,210] - [   SCORE    ] - Train: 

In [ ]:
model.fit(X_train, y_train, X_test, y_test, categorical_features=categorical_features, save_models=False, save_test=False)

[2024-11-05 15:30:53,139] - [   MODEL    ] - 1 out of 2. TabularLamaNN_autoint
[2024-11-05 15:30:53,143] - [   START    ] - Working with TabularLamaNN_autoint
[2024-11-05 15:30:53,144] - [   START    ] - Fitting TabularLamaNN_autoint
[15:30:53] Stdout logging level is INFO.
[15:30:53] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[15:30:53] Task: binary

[15:30:53] Start automl preset with listed constraints:
[15:30:53] - time: 600.00 seconds
[15:30:53] - CPU: 16 cores
[15:30:53] - memory: 16 GB

[15:30:53] Train data shape: (300000, 68)

[15:31:03] Layer 1 train process start. Time left 589.58 secs
[15:31:08] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_autoint_0 ...
[16:01:56] Time limit exceeded after calculating fold 0

[16:01:56] Fitting Lvl_0_Pipe_0_Mod_0_TorchNN_autoint_0 finished. score = 0.7997318221320568
[16:01:56] Lvl_0_Pipe_0_Mod_0_TorchNN_autoint_0 fitting and predicting completed
[16:01:56] Time left -1263.36 secs

[16:01:

In [15]:
model.best_model

In [16]:
model.best_model.params

{'objective_type': 'binary',
 'boosting': 'gbdt',
 'num_iterations': 284,
 'max_depth': 7,
 'learning_rate': 0.03,
 'num_leaves': 85,
 'min_data_in_leaf': 162,
 'bagging_fraction': 0.9073942790005392,
 'bagging_freq': 20,
 'feature_fraction': 0.4632002607000075,
 'early_stopping_round': 100,
 'lambda_l1': 0.9117760912120141,
 'lambda_l2': 6.182177721979992,
 'min_gain_to_split': 2.764898005468358,
 'num_threads': 16,
 'random_state': 77,
 'is_unbalance': False,
 'num_classes': 1,
 'verbose': -1}

## Inference best model

In [13]:
test = pd.read_parquet(DATA_PATH / "test_preproc.parquet")

In [19]:
test["target"] = model.best_model.predict(test[cfg["selected_features"] + oe_cols + ohe_cols])[:, 1]

In [21]:
test[['id', 'target']].to_csv('lgmb_oe_ohe_cols_0805.csv', index=False)

In [24]:
test["target"].describe()

count    171919.000000
mean          0.059860
std           0.067099
min           0.001254
25%           0.014259
50%           0.031264
75%           0.074442
max           0.489716
Name: target, dtype: float64